# Download snow class data for aso rasters 
This notebook contains functions to download and extract snow class data for ASO rasters

In [1]:
import sys
import os
from os.path import basename, exists, expanduser, join
import shutil
import shapely
import rioxarray as rxr
import xarray as xr
import time
import gzip
from urllib.request import urlretrieve
import matplotlib.pyplot as plt
import glob
import rasterio as rio

## Download fcf data

In [2]:
sc_path ='../../data/tmp/snow_class.tif'

## extract fcf data for aso rasters

In [3]:
def sc_for_aso(aso_raster_fn, sc_path):
    aso_raster = rxr.open_rasterio(aso_raster_fn).squeeze()
    aso_raster = aso_raster.where(aso_raster>=0, drop=True)
    aso_raster = aso_raster.interpolate_na(dim='x')
    
    # open as dataArray and return
    sc_ds = rxr.open_rasterio(sc_path)

    # clip first to avoid super long reproject processes
    sc_ds = sc_ds.rio.clip_box(*aso_raster.rio.transform_bounds("EPSG:4326"))
    # reproject FCF to match dataset
    sc_ds = sc_ds.rio.reproject_match(aso_raster, resampling=rio.enums.Resampling.nearest)
    # remove band dimension as it only has one band
    sc_ds = sc_ds.squeeze('band')
    sc_ds = sc_ds.rio.write_crs(aso_raster.rio.crs)
    
    sc_ds.to_netcdf(f'../../data/snow_class/sc_for_{aso_raster_fn.split("/")[-1][:-4]}.nc')

    #return fcf, aso_raster

In [4]:
def sc_for_aso_all(dir_path, sc_path):
    raster_paths = glob.glob(f'{dir_path}/*/ASO_50M_SD*.tif')
    for i, path in enumerate(raster_paths):
        print(f'----\nworking on {path.split("/")[-1]}, {i+1}/{len(raster_paths)}\n----')

        sc_for_aso(path, sc_path)

In [5]:
dir_path = '/mnt/c/Users/qbren/OneDrive - UW/Desktop/taco/projects/crunchy-snow/data/ASO/ASO_50m_SD_cleaned'
sc_for_aso_all(dir_path, sc_path)

----
working on ASO_50M_SD_American_20230131_clean.tif, 1/252
----
----
working on ASO_50M_SD_American_20230413_clean.tif, 2/252
----
----
working on ASO_50M_SD_American_20230428_clean.tif, 3/252
----
----
working on ASO_50M_SD_American_20230602_clean.tif, 4/252
----
----
working on ASO_50M_SD_Feather_20220310_clean.tif, 5/252
----
----
working on ASO_50M_SD_Feather_20220331_clean.tif, 6/252
----
----
working on ASO_50M_SD_Feather_20220429_clean.tif, 7/252
----
----
working on ASO_50M_SD_Feather_20230206_clean.tif, 8/252
----
----
working on ASO_50M_SD_Feather_20230409_clean.tif, 9/252
----
----
working on ASO_50M_SD_Feather_20230428_clean.tif, 10/252
----
----
working on ASO_50M_SD_Feather_20230618_clean.tif, 11/252
----
----
working on ASO_50M_SD_LowerPit_20230212_clean.tif, 12/252
----
----
working on ASO_50M_SD_LowerPit_20230425_clean.tif, 13/252
----
----
working on ASO_50M_SD_LowerPit_20230511_clean.tif, 14/252
----
----
working on ASO_50M_SD_SacramentoMcCloud_20230209_clean.tif,